In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM,BatchNormalization, Conv1D, MaxPooling1D , Flatten
from keras.layers.core import Dense, Dropout
from sklearn.metrics import accuracy_score

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [0]:
def data():
    ''' This function is to load the data'''
    SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/train/Inertial Signals/{signal}_train.txt'
        signals_data.append(
            pd.read_csv(filename, delim_whitespace=True, header=None).to_numpy()
        ) 
    X_train = np.transpose(signals_data, (1, 2, 0))
    
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/test/Inertial Signals/{signal}_test.txt'
        signals_data.append(
            pd.read_csv(filename, delim_whitespace=True, header=None).to_numpy()
        ) 
    X_test = np.transpose(signals_data, (1, 2, 0))
    
    filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/train/y_train.txt'
    y = pd.read_csv(filename, delim_whitespace=True, header=None)[0]
    Y_train = pd.get_dummies(y).to_numpy()
    
    filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/test/y_test.txt'
    y = pd.read_csv(filename, delim_whitespace=True, header=None)[0]
    Y_test = pd.get_dummies(y).to_numpy()
    
    return X_train , Y_train , X_test , Y_test

In [13]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 38kB/s 
     |████████████████████████████████| 3.2MB 51.8MB/s 
     |████████████████████████████████| 491kB 50.9MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0rc4
    Uninstalling tensorflow-2.2.0rc4:
      Successfully uninstalled tensorflow-2.2.0rc4


In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [0]:
# Initializing parameters
epochs = 30
batch_size = 32
n_hidden = 32

In [0]:
# Loading the train and test data
X_train, Y_train,X_test, Y_test = data()

In [10]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 6

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [12]:
# Initiliazing the sequential model
model_1 = Sequential()
# Configuring the parameters
model_1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_1.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model_1.add(Dense(n_classes, activation='sigmoid'))
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model_1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
# Training the model
model_1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 9s 1ms/step - loss: 1.3955 - accuracy: 0.4264 - val_loss: 1.2273 - val_accuracy: 0.4581
Epoch 2/30
7352/7352 [==============================] - 9s 1ms/step - loss: 1.0696 - accuracy: 0.5386 - val_loss: 1.0074 - val_accuracy: 0.5351
Epoch 3/30
7352/7352 [==============================] - 8s 1ms/step - loss: 0.9063 - accuracy: 0.6197 - val_loss: 0.8869 - val_accuracy: 0.6583
Epoch 4/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.7980 - accuracy: 0.6561 - val_loss: 0.7894 - val_accuracy: 0.6597
Epoch 5/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.7093 - accuracy: 0.6889 - val_loss: 0.7317 - val_accuracy: 0.6678
Epoch 6/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.6791 - accuracy: 0.6961 - val_loss: 0.8133 - val_accuracy: 

In [15]:
model_1.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 263us/step


[0.5119841206115033, 0.894808292388916]

In [16]:
# Confusion Matrix
print(confusion_matrix(Y_train, model_1.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                  77       97  ...                  62                91
SITTING                 58       41  ...                  77               133
STANDING                75       48  ...                 128                63
WALKING                 94      133  ...                  74                92
WALKING_DOWNSTAIRS     120       83  ...                  29                90
WALKING_UPSTAIRS        86       73  ...                  69                59

[6 rows x 6 columns]


We will be breaking the train data into static and dynamic activities. For that we change the labels of the train data as 1 for dynamic activities and 0 for static activities.



In [0]:
filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/train/y_train.txt'
y = pd.read_csv(filename, delim_whitespace=True, header=None)[0]
y_train = y.map({1:1,2:1, 3:1,\
                       4:0, 5:0,6:0})

In [0]:
filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/test/y_test.txt'
y = pd.read_csv(filename, delim_whitespace=True, header=None)[0]
y_test = y.map({1:1,2:1, 3:1,\
                       4:0, 5:0,6:0})

In [19]:
y_test.value_counts()

0    1560
1    1387
Name: 0, dtype: int64

In [20]:
#model to break the data into static and dynamic
model_s = Sequential()
model_s.add(LSTM(32, kernel_initializer='glorot_normal',input_shape=(128,9)))
model_s.add(Dropout(0.5))

model_s.add(Dense(1, activation='sigmoid',kernel_initializer='glorot_normal'))
model_s.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 5,409
Trainable params: 5,409
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_s.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
model_s.fit(X_train, y_train, batch_size=32, validation_data=(X_test, y_test),epochs=30)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.3545 - accuracy: 0.8289 - val_loss: 0.1898 - val_accuracy: 0.9494
Epoch 2/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.4493 - accuracy: 0.7716 - val_loss: 0.4969 - val_accuracy: 0.6787
Epoch 3/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.2120 - accuracy: 0.9251 - val_loss: 0.1396 - val_accuracy: 0.9647
Epoch 4/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.0880 - accuracy: 0.9814 - val_loss: 0.3946 - val_accuracy: 0.9274
Epoch 5/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.1928 - accuracy: 0.9445 - val_loss: 0.1454 - val_accuracy: 0.9572
Epoch 6/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.0851 - accuracy: 0.9800 - val_loss: 0.1038 - val_accuracy: 0.9766
Epoch 7/30
7352/7352 [==============================] - 9s 1ms/step - loss: 0.0721 - acc

In [0]:
y_pred_test = model_s.predict(X_test)
y_pred=[]
for i in y_pred_test:
    if i>0.5:                             #converting into 0 and 1 for the threshold 0.5
      y_pred.append(1)
    else:
      y_pred.append(0)

In [24]:
accuracy_score(y_test, y_pred)

0.99830335934849

In [0]:
def static_data():
    ''' this function breaks the train and test data on the basis of the static labels and stores them in files'''
    c=0
    k=0
    static = []
    sta = []
    static_y = []
    sta_y = []
    for i in y_train:
        if i==0:
            static.append(X_train[c])
            static_y.append(Y_train[c])
        c+=1
    for i in y_pred:
        if i==0:
            sta.append(X_test[k])
            sta_y.append(Y_test[k])
        k+=1
    X_tr_st = np.array(static)
    X_ts_st = np.array(sta)
    Y_tr_st = np.array(static_y)
    Y_ts_st = np.array(sta_y)

    np.save('X_tr_st', X_tr_st)
    np.save('Y_tr_st', Y_tr_st)
    np.save('X_ts_st', X_ts_st)
    np.save('Y_ts_st', Y_ts_st)
    return X_tr_st, Y_tr_st, X_ts_st, Y_ts_st

In [0]:
def dynamic_data():
    ''' this function breaks the train and test data on the basis of the dynamic labels and stores them in files '''
    c=0
    k=0
    dynamic=[]
    dynamic_y= []
    dyn=[]
    dyn_y= []

    for i in y_train:
        if i==1:
            dynamic.append(X_train[c])
            dynamic_y.append(Y_train[c])
        c+=1
    for i in y_pred:
        if i==1:
            dyn.append(X_test[k])
            dyn_y.append(Y_test[k])
        k+=1
    X_tr_dy = np.array(dynamic)
    X_ts_dy = np.array(dyn)
    Y_tr_dy = np.array(dynamic_y)
    Y_ts_dy = np.array(dyn_y)

    np.save('X_tr_dy', X_tr_dy)
    np.save('Y_tr_dy', Y_tr_dy)
    np.save('X_ts_dy', X_ts_dy)
    np.save('Y_ts_dy', Y_ts_dy)
    return X_tr_dy, Y_tr_dy, X_ts_dy, Y_ts_dy

In [0]:
X_tr_st, Y_tr_st, X_ts_st, Y_ts_st= static_data()      #Calling the static and dynamic data with the functions 
X_tr_dy, Y_tr_dy, X_ts_dy, Y_ts_dy = dynamic_data()

In [0]:
def load_static():
    X_tr_st = np.load('X_tr_st.npy')                      #Function to load the data from the saved files
    Y_tr_st = np.load('Y_tr_st.npy')
    X_ts_st = np.load('X_ts_st.npy')
    Y_ts_st = np.load('Y_ts_st.npy')

    return X_tr_st, Y_tr_st, X_ts_st, Y_ts_st

In [0]:
def load_dynamic():
    X_tr_dy = np.load('X_tr_dy.npy')                  #Function to load the data from the saved files
    Y_tr_dy = np.load('Y_tr_dy.npy')
    X_ts_dy = np.load('X_ts_dy.npy')
    Y_ts_dy = np.load('Y_ts_dy.npy')

    return X_tr_dy, Y_tr_dy, X_ts_dy, Y_ts_dy

Now we build models separately on the static data and dynamic data with different structures.

In [0]:
# model for dynamic activities
def best_hyperparameters_dy(ker1,ker2,drp,neurons):

    model = Sequential()
    # Configuring the parameters
    model.add(Conv1D(ker1, kernel_size=3, padding='same', input_shape=(128,9), activation='relu', kernel_initializer='glorot_normal', strides=1))
    model.add(MaxPooling1D())
    model.add(Dropout(drp))

    model.add(Conv1D(ker2, kernel_size=2, padding='same',activation='relu', kernel_initializer='glorot_normal', strides=1 ))

    model.add(Flatten())
    model.add(Dense(neurons, kernel_initializer='glorot_normal', activation='relu'))
    model.add(BatchNormalization())

    model.add(Dense(n_classes, activation='sigmoid',kernel_initializer='glorot_normal'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model


In [31]:
ker1 = [30,35]
ker2= [50,45]
neurons = [40,45]
drp = [0.5,0.6]
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=best_hyperparameters_dy, epochs=10, batch_size=batch_size, verbose=0)
param_grid = dict(ker1=ker1, ker2=ker2 ,neurons=neurons, drp=drp)

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result_dy = grid.fit(X_tr_dy, Y_tr_dy)

In [32]:
print("Best: %f using %s" % (grid_result_dy.best_score_, grid_result_dy.best_params_))
means = grid_result_dy.cv_results_['mean_test_score']
stds = grid_result_dy.cv_results_['std_test_score']
params = grid_result_dy.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.948250 using {'drp': 0.5, 'ker1': 30, 'ker2': 50, 'neurons': 40}
0.948250 (0.035304) with: {'drp': 0.5, 'ker1': 30, 'ker2': 50, 'neurons': 40}
0.927854 (0.046820) with: {'drp': 0.5, 'ker1': 30, 'ker2': 50, 'neurons': 45}
0.934855 (0.057914) with: {'drp': 0.5, 'ker1': 30, 'ker2': 45, 'neurons': 40}
0.927549 (0.043656) with: {'drp': 0.5, 'ker1': 30, 'ker2': 45, 'neurons': 45}
0.932725 (0.058961) with: {'drp': 0.5, 'ker1': 35, 'ker2': 50, 'neurons': 40}
0.922374 (0.058863) with: {'drp': 0.5, 'ker1': 35, 'ker2': 50, 'neurons': 45}
0.899239 (0.058803) with: {'drp': 0.5, 'ker1': 35, 'ker2': 45, 'neurons': 40}
0.931202 (0.056978) with: {'drp': 0.5, 'ker1': 35, 'ker2': 45, 'neurons': 45}
0.897412 (0.076083) with: {'drp': 0.6, 'ker1': 30, 'ker2': 50, 'neurons': 40}
0.922983 (0.079057) with: {'drp': 0.6, 'ker1': 30, 'ker2': 50, 'neurons': 45}
0.849011 (0.153164) with: {'drp': 0.6, 'ker1': 30, 'ker2': 45, 'neurons': 40}
0.930289 (0.038402) with: {'drp': 0.6, 'ker1': 30, 'ker2': 45, 'neuro

In [33]:
model_dy= best_hyperparameters_dy(30,50,0.5,40)
model_dy.fit(X_tr_dy, Y_tr_dy, epochs=30, batch_size=batch_size, validation_data= (X_ts_dy, Y_ts_dy))
model_dy.evaluate(X_ts_dy,Y_ts_dy)

Train on 3285 samples, validate on 1390 samples
Epoch 1/30
3285/3285 [==============================] - 4s 1ms/step - loss: 1.5592 - accuracy: 0.4377 - val_loss: 1.5287 - val_accuracy: 0.6115
Epoch 2/30
3285/3285 [==============================] - 1s 399us/step - loss: 1.0502 - accuracy: 0.7452 - val_loss: 0.8860 - val_accuracy: 0.7727
Epoch 3/30
3285/3285 [==============================] - 1s 402us/step - loss: 0.5494 - accuracy: 0.9008 - val_loss: 0.5669 - val_accuracy: 0.8360
Epoch 4/30
3285/3285 [==============================] - 1s 398us/step - loss: 0.2317 - accuracy: 0.9562 - val_loss: 0.3103 - val_accuracy: 0.9115
Epoch 5/30
3285/3285 [==============================] - 1s 406us/step - loss: 0.1130 - accuracy: 0.9784 - val_loss: 0.1692 - val_accuracy: 0.9504
Epoch 6/30
3285/3285 [==============================] - 1s 403us/step - loss: 0.0735 - accuracy: 0.9842 - val_loss: 0.2980 - val_accuracy: 0.8928
Epoch 7/30
3285/3285 [==============================] - 1s 396us/step - loss: 

[0.1227405796033538, 0.9676259160041809]

In [34]:
# Confusion Matrix
print(confusion_matrix(Y_ts_dy, model_dy.predict(X_ts_dy)))

Pred                WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                                             
SITTING                   0                   1                 1
STANDING                  1                   0                 1
WALKING                 489                   6                 1
WALKING_DOWNSTAIRS       14                 404                 1
WALKING_UPSTAIRS          6                  13               452


In [0]:
#Model for the static activities
def best_hyperparameters_st(k1,k2,drp,neurons):
  
    model = Sequential()
    # Configuring the parameters
    model.add(Conv1D(k1, kernel_size=2, padding='same', input_shape=(128,9), activation='relu', kernel_initializer='glorot_normal', strides=1))

    model.add(Conv1D(k2, kernel_size=3, padding='same', activation='relu', kernel_initializer='glorot_normal', strides=1))
    model.add(Dropout(drp))

    model.add(Flatten())

    model.add(Dense(neurons, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dropout(drp))
    model.add(BatchNormalization())

    model.add(Dense(n_classes, activation='sigmoid',kernel_initializer='glorot_normal'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

In [0]:
k1 = [40,32]
k2= [40,45]
neurons = [40,45]
drp = [0.5,0.6]

model = KerasClassifier(build_fn=best_hyperparameters_st, epochs=20, batch_size=batch_size, verbose=0)
param_grid = dict(k1=k1, k2=k2 ,neurons=neurons, drp=drp)

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result_st = grid.fit(X_tr_st, Y_tr_st)

In [37]:
print("Best: %f using %s" % (grid_result_st.best_score_, grid_result_st.best_params_))
means = grid_result_st.cv_results_['mean_test_score']
stds = grid_result_st.cv_results_['std_test_score']
params = grid_result_st.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.894275 using {'drp': 0.6, 'k1': 40, 'k2': 40, 'neurons': 45}
0.888371 (0.030247) with: {'drp': 0.5, 'k1': 40, 'k2': 40, 'neurons': 40}
0.880996 (0.023360) with: {'drp': 0.5, 'k1': 40, 'k2': 40, 'neurons': 45}
0.887884 (0.034688) with: {'drp': 0.5, 'k1': 40, 'k2': 45, 'neurons': 40}
0.861562 (0.040644) with: {'drp': 0.5, 'k1': 40, 'k2': 45, 'neurons': 45}
0.868947 (0.032568) with: {'drp': 0.5, 'k1': 32, 'k2': 40, 'neurons': 40}
0.878035 (0.023374) with: {'drp': 0.5, 'k1': 32, 'k2': 40, 'neurons': 45}
0.887384 (0.027393) with: {'drp': 0.5, 'k1': 32, 'k2': 45, 'neurons': 40}
0.878284 (0.029147) with: {'drp': 0.5, 'k1': 32, 'k2': 45, 'neurons': 45}
0.893290 (0.031109) with: {'drp': 0.6, 'k1': 40, 'k2': 40, 'neurons': 40}
0.894275 (0.019189) with: {'drp': 0.6, 'k1': 40, 'k2': 40, 'neurons': 45}
0.865744 (0.032084) with: {'drp': 0.6, 'k1': 40, 'k2': 45, 'neurons': 40}
0.856891 (0.034861) with: {'drp': 0.6, 'k1': 40, 'k2': 45, 'neurons': 45}
0.878787 (0.031900) with: {'drp': 0.6, 'k1'

In [41]:
model_static= best_hyperparameters_st(40,45,0.6,40)
model_static.fit(X_tr_st, Y_tr_st, epochs=30, batch_size=16, validation_data= (X_ts_st, Y_ts_st))
model_static.evaluate(X_ts_st,Y_ts_st)

Train on 4067 samples, validate on 1557 samples
Epoch 1/30
4067/4067 [==============================] - 8s 2ms/step - loss: 1.0652 - accuracy: 0.7750 - val_loss: 0.5312 - val_accuracy: 0.8728
Epoch 2/30
4067/4067 [==============================] - 3s 822us/step - loss: 0.4644 - accuracy: 0.8768 - val_loss: 0.3419 - val_accuracy: 0.8632
Epoch 3/30
4067/4067 [==============================] - 3s 858us/step - loss: 0.3113 - accuracy: 0.8866 - val_loss: 0.3475 - val_accuracy: 0.8760
Epoch 4/30
4067/4067 [==============================] - 3s 852us/step - loss: 0.2997 - accuracy: 0.8832 - val_loss: 0.4714 - val_accuracy: 0.8195
Epoch 5/30
4067/4067 [==============================] - 4s 893us/step - loss: 0.2899 - accuracy: 0.8852 - val_loss: 0.2906 - val_accuracy: 0.8767
Epoch 6/30
4067/4067 [==============================] - 3s 850us/step - loss: 0.3096 - accuracy: 0.8807 - val_loss: 0.3499 - val_accuracy: 0.8677
Epoch 7/30
4067/4067 [==============================] - 3s 814us/step - loss: 

[0.2652171851347499, 0.8805394768714905]

In [42]:
# Confusion Matrix
print(confusion_matrix(Y_ts_st, model_static.predict(X_ts_st)))

Pred                LAYING  SITTING  STANDING
True                                         
LAYING                 536        0         1
SITTING                  0      388       101
STANDING                 0       83       447
WALKING_DOWNSTAIRS       0        0         1


In [43]:
print('Mean accuracy over the static and dynamic data is:',np.add(96.76, 88.05 )/2.0)

Mean accuracy over the static and dynamic data is: 92.405


So we want a better accuracy using the same data building more complex models. So we use Divide and conquer method to build better models. We break our data into static and dynamic activity data using an LSTM model.As now we have separate statc and dynamic activity data we build independent models on these data using Conv1D and note the accuracy obtained . Using this method we land up having a better accuracy than just using LSTM model alone.

In [45]:
from prettytable import PrettyTable
x= PrettyTable(['Model','Accuracy'])
x.add_row(['Logistic Regression',96.27])
x.add_row(['Linear SVC',96.61])
x.add_row(['RBF SVM',96.27])
x.add_row(['Decision Tree',86.43])
x.add_row(['RandomForest',91.31])
x.add_row(['Gradient Boosted',91.31])
x.add_row(['LSTM',90.19])
x.add_row(['Divide and Conquer (Static)',88.05])
x.add_row(['Divide and Conquer (Dynamic)',96.70])
x.add_row(['Divide and Conquer avg',92.05])
print(x)

+------------------------------+----------+
|            Model             | Accuracy |
+------------------------------+----------+
|     Logistic Regression      |  96.27   |
|          Linear SVC          |  96.61   |
|           RBF SVM            |  96.27   |
|        Decision Tree         |  86.43   |
|         RandomForest         |  91.31   |
|       Gradient Boosted       |  91.31   |
|             LSTM             |  90.19   |
| Divide and Conquer (Static)  |  88.05   |
| Divide and Conquer (Dynamic) |   96.7   |
|    Divide and Conquer avg    |  92.05   |
+------------------------------+----------+
